In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mt
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


In [ ]:
read_data = pd.read_csv("/content/lab_vehicle.csv")
read_data.head()

,COMPACTNESS,CIRCULARITY,'DISTANCE CIRCULARITY','RADIUS RATIO','PR.AXIS ASPECT RATIO','MAX.LENGTH ASPECT RATIO','SCATTER RATIO',ELONGATEDNESS,'PR.AXIS RECTANGULARITY','MAX.LENGTH RECTANGULARITY','SCALED VARIANCE_MAJOR','SCALED VARIANCE_MINOR','SCALED RADIUS OF GYRATION','SKEWNESS ABOUT_MAJOR','SKEWNESS ABOUT_MINOR','KURTOSIS ABOUT_MAJOR','KURTOSIS ABOUT_MINOR','HOLLOWS RATIO'
0,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197
1,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199
2,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196
3,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207
4,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183


In [ ]:
x = read_data.shape

In [ ]:
def centroid(x, y):
  return np.mean(x), np.mean(y)

In [ ]:
def distance(x1, x2, y1, y2):
  return np.sqrt((x1-x2)**2 + (y1-y2)**2)

In [ ]:
def k_means(data, k, max_iterations=100): # Changed max_iterations to a fixed value
    # Initialize centroids randomly
    centroids = data.sample(n=k)
    print(k)

    for _ in range(max_iterations):
        # Assign data points to the nearest centroid
        labels = []
        for index, row in data.iterrows():
            distances = [distance(row['COMPACTNESS'], centroid_row['COMPACTNESS'], # Changed column names to uppercase
                                  row['CIRCULARITY'], centroid_row['CIRCULARITY']) # Changed column names to uppercase
                         for _, centroid_row in centroids.iterrows()]
            labels.append(np.argmin(distances))
        data['labels'] = labels

        # Update centroids based on the new cluster assignments
        new_centroids = data.groupby('labels').apply(lambda x: pd.Series({'COMPACTNESS': centroid(x['COMPACTNESS'], x['CIRCULARITY'])[0], # Changed column names to uppercase
                                                                         'CIRCULARITY': centroid(x['COMPACTNESS'], x['CIRCULARITY'])[1]})).reset_index() # Changed column names to uppercase

        # Check for convergence
        if centroids.equals(new_centroids):
            break
        centroids = new_centroids

    return data, centroids

In [ ]:
inertia = []

for k in range(1, 11):
    # Copy the relevant features
    data_copy = read_data[['COMPACTNESS', 'CIRCULARITY']].copy()

    # Run k-means clustering (ensure your k_means function returns labels and centroids)
    clustered_data, centroids = k_means(data_copy, k=k)

    # Extract feature values and labels
    features = data_copy.values
    labels = clustered_data['labels'].values
    cluster_centers = centroids[['COMPACTNESS', 'CIRCULARITY']].values

    # Calculate inertia manually: sum of squared distances to assigned centroids
    total_inertia = np.sum([np.sum((features[i] - cluster_centers[labels[i]]) ** 2) for i in range(len(features))])
    inertia.append(total_inertia)

# Plot the elbow curve
plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), inertia, marker='o', color='b')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.grid(True)
plt.show()

1


<ipython-input-158-58ef55f28dbe>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_centroids = data.groupby('labels').apply(lambda x: pd.Series({'COMPACTNESS': centroid(x['COMPACTNESS'], x['CIRCULARITY'])[0], # Changed column names to uppercase
<ipython-input-158-58ef55f28dbe>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_centroids = data.groupby('labels').apply(lambda x: pd.Series({'COMPACTNESS

ValueError: operands could not be broadcast together with shapes (3,) (2,) 

In [ ]:
data_for_clustering = read_data[['COMPACTNESS','CIRCULARITY']]
# data_for_clustering = data_for_clustering.rename(columns={'COMPACTNESS': 'compactness', 'CIRCULARITY': 'circularity'})
clustered_data, centroid = k_means(data_for_clustering, k=4)
print(clustered_data.head())
print(centroid)

4


<ipython-input-158-58ef55f28dbe>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['labels'] = labels
<ipython-input-158-58ef55f28dbe>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_centroids = data.groupby('labels').apply(lambda x: pd.Series({'COMPACTNESS': centroid(x['COMPACTNESS'], x['CIRCULARITY'])[0], # Changed column names to uppercase
<ipython-input-158-58ef55f28dbe>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

   COMPACTNESS  CIRCULARITY  labels
0           95           48       1
1           91           41       2
2          104           50       0
3           93           41       2
4           85           44       3
   labels  COMPACTNESS  CIRCULARITY
0       0   104.836449    53.000000
1       1    94.691099    46.591623
2       2    91.923469    38.704082
3       3    84.546939    41.330612


<ipython-input-158-58ef55f28dbe>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['labels'] = labels
<ipython-input-158-58ef55f28dbe>:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  new_centroids = data.groupby('labels').apply(lambda x: pd.Series({'COMPACTNESS': centroid(x['COMPACTNESS'], x['CIRCULARITY'])[0], # Changed column names to uppercase


In [ ]:
silhouette_score(clustered_data[['COMPACTNESS', 'CIRCULARITY']], labels = clustered_data['labels'])

np.float64(0.39199528806223666)